Import packages:

In [17]:
from finetune import Classifier, config
import tensorflow as tf
import csv
import pandas
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.20.0.


Get data:

In [18]:
filePath = "Processing_eda/Bloomington_311_processed_3.csv"
data3 = pandas.read_csv(filePath)
print(data3.shape)
print(data3.loc[82480])

(82484, 5)
service_name                                                 Yard Waste
description           Neighbor water buckets\n\nThe owner of the hou...
agency_responsible                                                  NaN
OurLabel                                                     Yard Waste
OurNumericLabel                                                       8
Name: 82480, dtype: object


Filter data only to ones with at least 40 characters and at most 1024

In [19]:
mask = (data3['description'].str.len() >=40) & (data3['description'].str.len() <=512)
dataFiltered = data3.loc[mask]
print(dataFiltered.shape)

(41374, 5)


Check for columns with NaN values

In [20]:
dataFiltered.columns[dataFiltered.isna().any()].tolist()
# ourLabel doesn't have NaN values, so that is good. 

['agency_responsible']

Prepare training data

In [22]:
trainingData = dataFiltered[["description", "OurLabel"]]
print(type(trainingData))
print(trainingData.shape)
trainX, testX, trainY, testY = train_test_split(trainingData.description, trainingData.OurLabel, test_size=0.2, random_state=42)
# bigMask = (trainingData["description"].str.len() >=1000)
# print(trainingData.loc[bigMask].shape)
# Split in train and test 80/20
print(trainX.shape)
print(type(trainX))
print(trainY.shape)

<class 'pandas.core.frame.DataFrame'>
(41374, 2)
(33099,)
<class 'pandas.core.series.Series'>
(33099,)


Training a model with finetune (this part fails because of CPU overload)

In [ ]:
model = Classifier(max_length=512, val_interval=3000, verbose = False)               # Load base model
model.fit(trainX, trainY)          # Finetune base model on custom data

Predictions

In [ ]:
predictions = model.predict(test) # [{'class_1': 0.23, 'class_2': 0.54, ..}, ..]
model.save("newModel")                   # Serialize the model to disk
